如果只用VEHICLE TYPE来决定的话，有的field是0，把==0和>3的加到一起就得到ppt里给的参考答案了。
但是比如第477691行，VEHICLE FACTOR比TYPE多，也说明至少两辆吧，还是说无视比较好

step 1 <br>
    1. 筛选有用的column - 意味着从这儿开始就得做俩表
    2. 除掉BOROUGH里的空行
    3. 选年份 year>=2015 //简单的检验了一下，什么都不用删
step2 <br>
    4. 每一行，查看vehicle type和 vehicle factor, 建两个series呗，然后比较大小，同一index哪个大取哪个，-1就是涉及到的车数
    5. 加一个column "NUMBER_OF_VECHICLES_INVOLVED"
step3 <br>
    6. 根据"NUMBER_OF_VECHICLES_INVOLVED"的数值创建新的columns ("ONE_VEHICLE_INVOLVED","TWO_VEHICLES_INVOLVED","THREE_VEHICLES_INVOLVED","MORE_VEHICLES_INVOLVED")
    7. 最后groupby"BOROUGH"一下

In [87]:
from pandas import Series, DataFrame
import pandas as pd
import os
import numpy as np

In [2]:
df_origin = pd.read_csv(os.getcwd() + '/Data/vehicle_collisions.csv')
df_origin.head(5)

,UNIQUE KEY,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE,VEHICLE 1 FACTOR,VEHICLE 2 FACTOR,VEHICLE 3 FACTOR,VEHICLE 4 FACTOR,VEHICLE 5 FACTOR
0,3146911,1/1/15,0:20,QUEENS,11358.0,40.751847,-73.787862,"(40.7518471, -73.787862)",47 AVENUE,193 STREET,...,SPORT UTILITY/STATION WAGON,NaN,NaN,NaN,NaN,TRAFFIC CONTROL DISREGARDED,NaN,NaN,NaN,NaN
1,3146180,1/1/15,0:20,NaN,NaN,40.771289,-73.946693,"(40.7712888, -73.9466928)",NaN,NaN,...,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,ANIMALS ACTION,NaN,NaN,NaN,NaN
2,3146384,1/1/15,0:21,BROOKLYN,11205.0,40.689445,-73.955121,"(40.6894449, -73.9551212)",BEDFORD AVENUE,LAFAYETTE AVENUE,...,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,NaN
3,3146013,1/1/15,0:30,BROOKLYN,11213.0,40.673845,-73.925080,"(40.6738445, -73.9250801)",BUFFALO AVENUE,SAINT MARKS AVENUE,...,BUS,PASSENGER VEHICLE,NaN,NaN,NaN,LOST CONSCIOUSNESS,NaN,NaN,NaN,NaN
4,3146120,1/1/15,0:33,NaN,NaN,NaN,NaN,NaN,RICHMOND TERRACE,SOUTH AVENUE,...,UNKNOWN,PASSENGER VEHICLE,NaN,NaN,NaN,UNSPECIFIED,UNSPECIFIED,NaN,NaN,NaN


In [267]:
df_origin.ix[477691]

UNIQUE KEY                                 3624019
DATE                                       2/28/17
TIME                                          8:50
BOROUGH                                   BROOKLYN
ZIP CODE                                     11226
LATITUDE                                   40.6504
LONGITUDE                                 -73.9625
LOCATION                      (40.65038, -73.9625)
ON STREET NAME                                 NaN
CROSS STREET NAME                              NaN
OFF STREET NAME        136       SAINT PAULS PLACE
PERSONS INJURED                                  0
PERSONS KILLED                                   0
PEDESTRIANS INJURED                              0
PEDESTRIANS KILLED                               0
CYCLISTS INJURED                                 0
CYCLISTS KILLED                                  0
MOTORISTS INJURED                                0
MOTORISTS KILLED                                 0
VEHICLE 1 TYPE         SPORT UT

In [193]:
df_filter = df_origin[df_origin['BOROUGH'].notnull()]
df_filter.head(3)

,UNIQUE KEY,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE,VEHICLE 1 FACTOR,VEHICLE 2 FACTOR,VEHICLE 3 FACTOR,VEHICLE 4 FACTOR,VEHICLE 5 FACTOR
0,3146911,1/1/15,0:20,QUEENS,11358.0,40.751847,-73.787862,"(40.7518471, -73.787862)",47 AVENUE,193 STREET,...,SPORT UTILITY/STATION WAGON,NaN,NaN,NaN,NaN,TRAFFIC CONTROL DISREGARDED,NaN,NaN,NaN,NaN
2,3146384,1/1/15,0:21,BROOKLYN,11205.0,40.689445,-73.955121,"(40.6894449, -73.9551212)",BEDFORD AVENUE,LAFAYETTE AVENUE,...,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,NaN
3,3146013,1/1/15,0:30,BROOKLYN,11213.0,40.673845,-73.925080,"(40.6738445, -73.9250801)",BUFFALO AVENUE,SAINT MARKS AVENUE,...,BUS,PASSENGER VEHICLE,NaN,NaN,NaN,LOST CONSCIOUSNESS,NaN,NaN,NaN,NaN


In [194]:
df_filter1 = df_filter[['DATE','BOROUGH','VEHICLE 1 TYPE','VEHICLE 2 TYPE','VEHICLE 3 TYPE','VEHICLE 4 TYPE','VEHICLE 5 TYPE']]
df_filter1.head(5)

,DATE,BOROUGH,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE
0,1/1/15,QUEENS,SPORT UTILITY/STATION WAGON,NaN,NaN,NaN,NaN
2,1/1/15,BROOKLYN,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN
3,1/1/15,BROOKLYN,BUS,PASSENGER VEHICLE,NaN,NaN,NaN
5,1/1/15,BROOKLYN,PASSENGER VEHICLE,SPORT UTILITY/STATION WAGON,SPORT UTILITY/STATION WAGON,PASSENGER VEHICLE,NaN
6,1/1/15,QUEENS,PASSENGER VEHICLE,PASSENGER VEHICLE,SPORT UTILITY/STATION WAGON,NaN,NaN


In [195]:
df_filter2 =df_filter[['DATE','BOROUGH','VEHICLE 1 FACTOR','VEHICLE 2 FACTOR','VEHICLE 3 FACTOR','VEHICLE 4 FACTOR','VEHICLE 5 FACTOR']]
df_filter2.head(5)

,DATE,BOROUGH,VEHICLE 1 FACTOR,VEHICLE 2 FACTOR,VEHICLE 3 FACTOR,VEHICLE 4 FACTOR,VEHICLE 5 FACTOR
0,1/1/15,QUEENS,TRAFFIC CONTROL DISREGARDED,NaN,NaN,NaN,NaN
2,1/1/15,BROOKLYN,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,NaN
3,1/1/15,BROOKLYN,LOST CONSCIOUSNESS,NaN,NaN,NaN,NaN
5,1/1/15,BROOKLYN,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,NaN
6,1/1/15,QUEENS,ALCOHOL INVOLVEMENT,UNSPECIFIED,UNSPECIFIED,NaN,NaN


In [196]:
pd.to_datetime(df_filter1['DATE']).dt.year.unique()

array([2015, 2016, 2017])

In [198]:
pd.to_datetime(df_filter2['DATE']).dt.year.unique()

array([2015, 2016, 2017])

In [199]:
del df_filter1['DATE']
df_filter1.head(5)

,BOROUGH,VEHICLE 1 TYPE,VEHICLE 2 TYPE,VEHICLE 3 TYPE,VEHICLE 4 TYPE,VEHICLE 5 TYPE
0,QUEENS,SPORT UTILITY/STATION WAGON,NaN,NaN,NaN,NaN
2,BROOKLYN,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN
3,BROOKLYN,BUS,PASSENGER VEHICLE,NaN,NaN,NaN
5,BROOKLYN,PASSENGER VEHICLE,SPORT UTILITY/STATION WAGON,SPORT UTILITY/STATION WAGON,PASSENGER VEHICLE,NaN
6,QUEENS,PASSENGER VEHICLE,PASSENGER VEHICLE,SPORT UTILITY/STATION WAGON,NaN,NaN


In [200]:
del df_filter2['DATE']
df_filter2.head(5)

,BOROUGH,VEHICLE 1 FACTOR,VEHICLE 2 FACTOR,VEHICLE 3 FACTOR,VEHICLE 4 FACTOR,VEHICLE 5 FACTOR
0,QUEENS,TRAFFIC CONTROL DISREGARDED,NaN,NaN,NaN,NaN
2,BROOKLYN,FATIGUED/DROWSY,UNSPECIFIED,NaN,NaN,NaN
3,BROOKLYN,LOST CONSCIOUSNESS,NaN,NaN,NaN,NaN
5,BROOKLYN,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,NaN
6,QUEENS,ALCOHOL INVOLVEMENT,UNSPECIFIED,UNSPECIFIED,NaN,NaN


In [250]:
series_count = df_filter1.T.notnull().sum() -1

In [210]:
series_count = np.maximum(df_filter1.T.notnull().sum(),df_filter2.T.notnull().sum()) - 1

In [230]:
series_count.head(5)

0    1
2    2
3    2
5    4
6    3
dtype: int64

In [226]:
df_filter1.T.notnull().sum().head(5)

0    2
2    3
3    3
5    5
6    4
dtype: int64

In [227]:
df_filter2.T.notnull().sum().head(5)

0    2
2    3
3    2
5    5
6    4
dtype: int64

In [277]:
bools = df_filter1.T.notnull().sum()<df_filter2.T.notnull().sum()
bools.any()

True

In [217]:
df_step2 = df_filter1.drop(df_filter1.columns[[1,2,3,4,5]], axis=1)

In [256]:
df_step2 = df_filter1.drop(df_filter1.columns[[1,2,3,4,5]], axis=1)

In [257]:
df_step2['NUMBER_OF_VECHICLES_INVOLVED'] = series_count
df_step2.head(5)

,BOROUGH,NUMBER_OF_VECHICLES_INVOLVED
0,QUEENS,1
2,BROOKLYN,2
3,BROOKLYN,2
5,BROOKLYN,4
6,QUEENS,3


In [258]:
df_step3 = df_step2.copy()

In [259]:
df_step3['ONE_VEHICLE_INVOLVED'] = df_step3['NUMBER_OF_VECHICLES_INVOLVED']==1
df_step3.head(5)

,BOROUGH,NUMBER_OF_VECHICLES_INVOLVED,ONE_VEHICLE_INVOLVED
0,QUEENS,1,True
2,BROOKLYN,2,False
3,BROOKLYN,2,False
5,BROOKLYN,4,False
6,QUEENS,3,False


In [260]:
df_step3['TWO_VEHICLES_INVOLVED'] = df_step3['NUMBER_OF_VECHICLES_INVOLVED']==2
df_step3.head(5)

,BOROUGH,NUMBER_OF_VECHICLES_INVOLVED,ONE_VEHICLE_INVOLVED,TWO_VEHICLES_INVOLVED
0,QUEENS,1,True,False
2,BROOKLYN,2,False,True
3,BROOKLYN,2,False,True
5,BROOKLYN,4,False,False
6,QUEENS,3,False,False


In [261]:
df_step3['THREE_VEHICLES_INVOLVED'] = df_step3['NUMBER_OF_VECHICLES_INVOLVED']==3
df_step3.head(5)

,BOROUGH,NUMBER_OF_VECHICLES_INVOLVED,ONE_VEHICLE_INVOLVED,TWO_VEHICLES_INVOLVED,THREE_VEHICLES_INVOLVED
0,QUEENS,1,True,False,False
2,BROOKLYN,2,False,True,False
3,BROOKLYN,2,False,True,False
5,BROOKLYN,4,False,False,False
6,QUEENS,3,False,False,True


In [262]:
df_step3['MORE_VEHICLES_INVOLVED'] = df_step3['NUMBER_OF_VECHICLES_INVOLVED'] > 3
df_step3.head(5)

,BOROUGH,NUMBER_OF_VECHICLES_INVOLVED,ONE_VEHICLE_INVOLVED,TWO_VEHICLES_INVOLVED,THREE_VEHICLES_INVOLVED,MORE_VEHICLES_INVOLVED
0,QUEENS,1,True,False,False,False
2,BROOKLYN,2,False,True,False,False
3,BROOKLYN,2,False,True,False,False
5,BROOKLYN,4,False,False,False,True
6,QUEENS,3,False,False,True,False


In [263]:
df_step3['ZERO_VEHICLE_INVOLVED'] = df_step3['NUMBER_OF_VECHICLES_INVOLVED'] == 0
df_step3.head(5)

,BOROUGH,NUMBER_OF_VECHICLES_INVOLVED,ONE_VEHICLE_INVOLVED,TWO_VEHICLES_INVOLVED,THREE_VEHICLES_INVOLVED,MORE_VEHICLES_INVOLVED,ZERO_VEHICLE_INVOLVED
0,QUEENS,1,True,False,False,False,False
2,BROOKLYN,2,False,True,False,False,False
3,BROOKLYN,2,False,True,False,False,False
5,BROOKLYN,4,False,False,False,True,False
6,QUEENS,3,False,False,True,False,False


In [264]:
df_step3.groupby('BOROUGH').sum()


,NUMBER_OF_VECHICLES_INVOLVED,ONE_VEHICLE_INVOLVED,TWO_VEHICLES_INVOLVED,THREE_VEHICLES_INVOLVED,MORE_VEHICLES_INVOLVED,ZERO_VEHICLE_INVOLVED
BOROUGH,,,,,,
BRONX,85956,8627.0,34385.0,1962.0,625.0,525.0
BROOKLYN,199638,17375.0,80207.0,4980.0,1612.0,787.0
MANHATTAN,155439,13318.0,66958.0,2024.0,501.0,801.0
QUEENS,172970,12962.0,70260.0,4498.0,1420.0,515.0
STATEN ISLAND,26664,2343.0,10676.0,759.0,165.0,105.0
